In [58]:
import requests # from urllib.request import urlopen
from bs4 import BeautifulSoup # HTTP Response -> HTML 

import pandas as pd

from datetime import datetime

import time 
import re #정규 표현식 활용

In [59]:
#망고플레이트에서 음식을 키워드로 맛집 리스트 가져오는 function
def get_rest_list(keyword):
    
    #페이지, 맛집리스트 초기화
    pagenum = 1
    restaurant_list = []
    
    #page별로 while 루프 돌려서 상호명 뽑아오기
    while pagenum <= 2:
        try:
            url = 'https://www.mangoplate.com/search/?keyword='+keyword+'&page='+str(pagenum)

            session = requests.Session()
            headers = {
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit 537.36 (KHTML, like Gecko) Chrome",
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
            }
            html = session.get(url, headers=headers).content 
            source = BeautifulSoup(html, 'html.parser')

            restaurants = source.find_all('h2', {'class' : 'title'}) #h2태그의 class 'title' => 상호명

            for restaurant in restaurants:

                n_restaurant = re.sub('\s', '', restaurant.get_text()) #텍스트 중간에 빈칸 없애기
                restaurant_list.append(n_restaurant)
        except:
            print(pagenum,'페이지 까지의 결과입니다.')
        
        pagenum += 1
        
    return restaurant_list

In [60]:
cnoodlelist = get_rest_list('냉면')

In [61]:
cnoodlelist

['허브족발(본점)',
 '윤밀원',
 '김진목삼',
 '진미평양냉면',
 '을지면옥',
 '서관면옥',
 '광릉한옥집',
 '광화문국밥',
 '춘향미엔',
 '우래옥(을지로점)',
 '설눈',
 '봉밀가',
 '봉피양(방이점)',
 '미엔아이',
 '윤선희의평양냉면양각도',
 '청기와타운',
 '서평면옥',
 '몽탄',
 '정인면옥',
 '강서면옥',
 '오장동흥남집(본점)',
 '봉래면옥',
 '거대갈비',
 '평안도상원냉면',
 '명지원이동갈비',
 '오복함흥냉면',
 '벽제갈비(신촌점)',
 '공단떡볶이',
 '고명집',
 '우밀면옥',
 '단천식당',
 '명인등심(압구정점)',
 '평남면옥',
 '서정가야밀면',
 '더진미평냉',
 '명인등심(삼성직영점)',
 '영동냉면',
 '을밀대(본점)',
 '능라도(강남점)',
 '능라도(분당점)']

In [79]:
def get_mango_info(restaurant):

    mango_info = {}

    url = 'https://www.mangoplate.com/search/?keyword='+restaurant

    session = requests.Session()
    headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit 537.36 (KHTML, like Gecko) Chrome",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
    }
    html = session.get(url, headers=headers).content 
    source = BeautifulSoup(html, 'html.parser')

    rest_code = source.find('a',{'class':'only-desktop_not'}).attrs['href']
    newurl = 'https://www.mangoplate.com'+rest_code
    newhtml = session.get(newurl, headers=headers).content 
    newsource = BeautifulSoup(newhtml, 'html.parser')
    
    rate = float(re.sub('\s','',newsource.find('strong',{'class':'rate-point'}).get_text()))

    rest_table = newsource.find('table',{'class':'info'})

    address = rest_table.find_all('td')[0].find_all('span')[1].get_text()
    phone = rest_table.find_all('td')[1].get_text()
    kind = re.sub('\s','',rest_table.find_all('td')[2].get_text())
    price = rest_table.find_all('td')[3].get_text()

    mango_info['name'] = restaurant
    mango_info['rate'] = rate
    mango_info['address'] = address
    mango_info['phone'] = phone
    mango_info['kind'] = kind
    mango_info['price'] = price
    
    return mango_info



In [80]:
mango_name = []
mango_rate = []
mango_address = []
mango_phone = []
mango_kind = []
mango_price = []

for restaurant in cnoodlelist:
    mango_info = get_mango_info(restaurant)
    mango_name.append(mango_info['name'])
    mango_rate.append(mango_info['rate'])
    mango_address.append(mango_info['address'])
    mango_phone.append(mango_info['phone'])
    mango_kind.append(mango_info['kind'])
    mango_price.append(mango_info['price'])

    

In [81]:
mango_df = pd.DataFrame({'음식점':mango_name,
                         '평점': mango_rate,
                         '주소': mango_address,
                         '전화': mango_phone,
                         '종류': mango_kind,
                         '가격': mango_price})
mango_df.to_excel('result_{}.xlsx'.format(datetime.now().strftime('%y%m%d_%H%M')), index=False, encoding='utf-8')

mango_df

,음식점,평점,주소,전화,종류,가격
0,허브족발(본점),4.6,서울시 영등포구 당산동6가 237-38,02-2633-4339,고기요리,3만원-4만원
1,윤밀원,4.6,경기도 성남시 분당구 정자동 223-9,031-714-8388,국수/면요리,만원 미만
2,김진목삼,4.6,서울시 종로구 필운동 154,02-929-2929,고기요리,만원-2만원
3,진미평양냉면,4.5,서울시 강남구 논현동 115-10,02-515-3469,국수/면요리,만원-2만원
4,을지면옥,4.5,서울시 중구 입정동 161,02-2266-7052,국수/면요리,만원-2만원
5,서관면옥,4.5,서울시 서초구 서초동 1675-5,02-521-9945,국수/면요리,만원-2만원
6,광릉한옥집,4.5,경기도 남양주시 진접읍 부평리 602-1,031-574-6630,고기요리,만원-2만원
7,광화문국밥,4.5,서울시 중구 정동 1-48,02-738-5688,탕/찌개/전골,만원-2만원
8,춘향미엔,4.5,서울시 성동구 행당동 299-1,02-2281-9031,딤섬/만두,만원 미만
9,우래옥(을지로점),4.4,서울시 중구 주교동 118-1,02-2265-0151,국수/면요리,만원-2만원
